<a href="https://colab.research.google.com/github/1013115/KTB_Project/blob/main/KTB_Day6_GenAI_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! kaggle datasets download -d rkiattisak/traveler-trip-data

Dataset URL: https://www.kaggle.com/datasets/rkiattisak/traveler-trip-data
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/4.21k [00:00<?, ?B/s]
100% 4.21k/4.21k [00:00<00:00, 6.64MB/s]


In [6]:
! unzip /content/traveler-trip-data.zip

Archive:  /content/traveler-trip-data.zip
  inflating: Travel details dataset.csv  


In [10]:
import pandas as pd

data = pd.read_csv('/content/Travel details dataset.csv')

print("#"*10)
print("데이터의 처음 몇 줄을 출력하여 구조 확인")
print(data.head())

print("#"*10)
print("데이터의 각 칼럼에 대한 정보 확인")
print(data.info())

##########
데이터의 처음 몇 줄을 출력하여 구조 확인
   Trip ID       Destination Start date   End date  Duration (days)  \
0        1        London, UK   5/1/2023   5/8/2023              7.0   
1        2  Phuket, Thailand  6/15/2023  6/20/2023              5.0   
2        3   Bali, Indonesia   7/1/2023   7/8/2023              7.0   
3        4     New York, USA  8/15/2023  8/29/2023             14.0   
4        5      Tokyo, Japan  9/10/2023  9/17/2023              7.0   

   Traveler name  Traveler age Traveler gender Traveler nationality  \
0     John Smith          35.0            Male             American   
1       Jane Doe          28.0          Female             Canadian   
2      David Lee          45.0            Male               Korean   
3  Sarah Johnson          29.0          Female              British   
4     Kim Nguyen          26.0          Female           Vietnamese   

  Accommodation type Accommodation cost Transportation type  \
0              Hotel               1200         

In [11]:
print("#"*10)
print("데이터 타입 확인")
print("Data Types:\n", data.dtypes)

categorical_cols = data.select_dtypes(include=['object', 'category']).columns
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns

print("#"*10)
print("범주형 데이터 분리하여 분석")
print("\nCategorical Columns:\n", categorical_cols)

print("#"*10)
print("수치형 데이터 분리하여 분석")
print("\nNumerical Columns:\n", numerical_cols)

##########
데이터 타입 확인
Data Types:
 Trip ID                   int64
Destination              object
Start date               object
End date                 object
Duration (days)         float64
Traveler name            object
Traveler age            float64
Traveler gender          object
Traveler nationality     object
Accommodation type       object
Accommodation cost       object
Transportation type      object
Transportation cost      object
dtype: object
##########
범주형 데이터 분리하여 분석

Categorical Columns:
 Index(['Destination', 'Start date', 'End date', 'Traveler name',
       'Traveler gender', 'Traveler nationality', 'Accommodation type',
       'Accommodation cost', 'Transportation type', 'Transportation cost'],
      dtype='object')
##########
수치형 데이터 분리하여 분석

Numerical Columns:
 Index(['Trip ID', 'Duration (days)', 'Traveler age'], dtype='object')


In [12]:
# 결측치 확인
print("\nMissing Values:\n", data.isnull().sum())

# 각 범주형 변수의 유니크한 값과 빈도수
# 변수의 크기에 따라서도 과적합 등이 발생할 수 있음
for col in categorical_cols:
    print(f"\nUnique values in {col}:\n", data[col].value_counts())

# 수치형 데이터의 기초 통계
print("\nDescriptive Statistics for Numerical Data:\n", data[numerical_cols].describe())


Missing Values:
 Trip ID                 0
Destination             2
Start date              2
End date                2
Duration (days)         2
Traveler name           2
Traveler age            2
Traveler gender         2
Traveler nationality    2
Accommodation type      2
Accommodation cost      2
Transportation type     3
Transportation cost     3
dtype: int64

Unique values in Destination:
 Destination
Paris                          7
Bali                           7
Tokyo, Japan                   7
Paris, France                  7
Tokyo                          5
Sydney                         5
Rome                           5
New York                       5
Bali, Indonesia                5
Rio de Janeiro, Brazil         4
Bangkok                        4
Cancun, Mexico                 4
Barcelona, Spain               4
Rome, Italy                    4
London                         4
Bangkok, Thailand              3
Sydney, Aus                    3
London, UK                

In [15]:
# 왜도와 첨도 확인
# 정규 분포 관련 값이라 수치형 데이터를 넣어야함, 범주형을 넣으면 오류가 발생함
"""
왜도(Skewness): 0에 가까울수록 정규분포에 근사, 양의 값은 오른쪽 꼬리가 긴 분포(왼쪽으로 치우친), 음의 값은 왼쪽 꼬리가 긴 분포(오른쪽으로 치우친)
첨도(Kurtosis): 0에 가까울수록 정규분포에 근사, 높으면 분포가 뾰족하고, 낮으면 평평
"""

print("\nSkewness of the data:\n", data[numerical_cols].skew())
print("\nKurtosis of the data:\n", data[numerical_cols].kurt())


Skewness of the data:
 Trip ID            0.000000
Duration (days)    0.982737
Traveler age       0.971157
dtype: float64

Kurtosis of the data:
 Trip ID           -1.200000
Duration (days)    1.778925
Traveler age       0.820152
dtype: float64


In [20]:
from scipy import stats
"""
T-통계량의 절대값이 크면 클수록 두 그룹 간의 차이가 크다고 할 수 있습니다.

일반적으로 P-값이 0.05보다 작으면 귀무 가설을 기각하고, 통계적으로 유의미한 차이가 있음을 인정합니다
"""

# 남성과 여성 그룹 데이터 분할
male_age = data[data['Traveler gender'] == 'Male']['Traveler age']
female_age = data[data['Traveler gender'] == 'Female']['Traveler age']

# 두 그룹 간의 평균 최대 심박수 차이 검정
t_stat, p_val = stats.ttest_ind(male_age, female_age)
print(f"T-statistic: {t_stat}, P-value: {p_val}")

T-statistic: 5.20300976324119, P-value: 7.117679024236941e-07
